## Logger

In [8]:
"""Modulo per la gestione dei log dell'applicazione."""

# Standard Import
import logging

# Site-package Import
# Project Import


class LoggerManager:
    ''' Creo la classe Logger per creare un logger separato per ogni modulo.
        Occorre passare il nome del modulo e l'eventuale nomefile.
        Se non viene passato un nomefile, si utiizza lo standard.

        NOTSET     0  Non impostato
        DEBUG     10  Livello di Debug
        INFO      20  Livello di Informazione da mettere in output
        WARNING   30  Warning non bloccante
        ERROR     40  Errore
        CRITICAL  50  Errore critico bloccante
    '''

    def __init__(self, name, log_file=None,
                 base_log_level=1, file_log_level=None, console_log_level=None):
        
        self.logger = logging.getLogger(name)
        
        # Verifica livello BASE del Debug: standard = 1: passa tutto
        try:
            if not(0 <= base_log_level <= 50):
                base_log_level = 1
        except:
            base_log_level = 1

        # Verifica livello Debug per File: standard = 20: INFO
        try:
            if not(0 <= file_log_level <= 50):
                file_log_level = 20
        except:
            file_log_level = 20
        
        # Verifica livello Log per Console: standard = 10: DEBUG
        try:
            if not(0 <= console_log_level <= 50):
                console_log_level = 10
        except:
            console_log_level = 10

        
        # Imposto il livello alla base dei log che vengon passati agli Handler
        self.logger.setLevel(base_log_level)

        # imposto i 2 formatter per il File (DateTime) e per la Consolle (no DateTime)
        file_formatter = logging.Formatter("%(asctime)s %(name)s > %(levelname)-9s%(message)s", datefmt="%Y.%m.%d %H:%M:%S")
        cons_formatter = logging.Formatter("%(name)s > %(levelname)-9s%(message)s")    

        if log_file:
            ''' se viene passato un log_file creo l'Handler altrimenti solo su console'''
            file_handler = logging.FileHandler(log_file)
            file_handler.setLevel(file_log_level)
            file_handler.setFormatter(file_formatter)
            self.logger.addHandler(file_handler)
        
        ''' sulla Console imposto sempre il Logging, poi valutiamo insieme'''
        console_handler = logging.StreamHandler()
        console_handler.setLevel(console_log_level)
        console_handler.setFormatter(cons_formatter)
        self.logger.addHandler(console_handler)
    
    # override dei messaggi di Output del Logging
    def debug(self, message):
        self.logger.debug(message)
        
    def info(self, message):
        self.logger.info(message)
        
    def warning(self, message):
        self.logger.warning(message)
        
    def error(self, message):
        self.logger.error(message)
        
    def critical(self, message):
        self.logger.critical(message)



## Config 

In [12]:
from configparser import ConfigParser

test_config = ConfigParser()
test_config.read(filenames='./data/test.ini')
test_config['path'].get('source_folder')

'./data/source_folder'

## Import per il digger

In [53]:
from dataclasses import dataclass as dc, field 
import os
from hashlib import md5

## Data Class 'File'

In [2]:
@dc 
class File:
    name: str
    path: os.PathLike
    checksum: bytes = field(init=False)

    def __post_init__(self):
        with open(self.path, 'rb') as file:
            read_file = file.read()
            self.checksum = md5(read_file).digest()
    
            

In [3]:
!touch data/source_folder/test_file.txt

test01 = File('test_file.txt', './data/source_folder/test_file.txt')

print(test01)

print(test01.checksum)

File(name='test_file.txt', path='./data/source_folder/test_file.txt', checksum=b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x98\xec\xf8B~')
b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x98\xec\xf8B~'


## ***Digger***

In [86]:
class Digger: 

    __dirPath: os.path = '.'

    def __init__(self, config, log:LoggerManager, index=None) -> None:
        self.__dirPath = config["path"]["source_folder"]
        self.log = log 
        self.files = []
        self.index = index

    def scan(self, path=__dirPath) -> None:
        with os.scandir(path) as el:
            for i in el:
                if (i.is_file()):
                    self.log.info(f'File esaminato {i.path}')
                    
                    h = self.__resolve_checksum(i.path)
                    self.__update_index(i.path, h)


                elif(i.is_dir()):
                    self.scan(path=i)


    def __resolve_checksum(self, path, type='md5') -> bytes:
        """calcola il checksum del file al percorso indicato
        
        Return:
            binario che rappresenta il checksum (MD5)
        """
    
        try:
            with open(path, 'rb') as file:
                file_hash = md5(file.read())
                print(f'updated md5 {file_hash.digest()}')
                return file_hash.digest()
        except Exception as err:
            self.log.error(err)

    def __update_index(self, path, hash_bin) -> None:
        """cerca se il file è presente nel db e controlla il suo checksum
        se non presente lo inserisce"""

        """da implementare chiamate ai metodi di index"""
        if(self.__alredy_indexed(path)):
            print('controlla checksum e aggiorna')
            print(f'hash da confrontare {hash_bin}')
        else:
            print('inserisci nel db')

    def __alredy_indexed(self, path) -> bool:
        """interroga index, torna True se il file è già indicizzato, altrimenti False"""

        """da implementare chiamate ai metodi di index"""
        print(self.index)
        return True

## Test 

In [64]:
#Gestione errori apertura file con with
try:
    with open('./data/database.txt', 'rb') as test_errore:
        h = md5(test_errore.read())
        print(h.digest())

except Exception as err:
    print(f'Loggare errore {err} ')

b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x98\xec\xf8B~'


In [87]:
test_log = LoggerManager('test', log_file='./data/test_log.txt')

dig = Digger(test_config, test_log)
dig.scan()

test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INFO     File esaminato ./data/elaborated_folder/README.md
test > INF

updated md5 b'\x88\xb9\x80\x076\xb6\x19%:\xc2\x1d\xe0\x8cl\x84\x06'
None
controlla checksum e aggiorna
hash da confrontare b'\x88\xb9\x80\x076\xb6\x19%:\xc2\x1d\xe0\x8cl\x84\x06'
updated md5 b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x98\xec\xf8B~'
None
controlla checksum e aggiorna
hash da confrontare b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x98\xec\xf8B~'
updated md5 b'\xaaG\xd9\xab\xcc\x8f>\xf1\xfe\xbaB#!8\xc9\x88'
None
controlla checksum e aggiorna
hash da confrontare b'\xaaG\xd9\xab\xcc\x8f>\xf1\xfe\xbaB#!8\xc9\x88'
updated md5 b"'\xab\xe59\x0bj\xd1X\xc2@\xab\xc5\xa2\x18\x1d>"
None
controlla checksum e aggiorna
hash da confrontare b"'\xab\xe59\x0bj\xd1X\xc2@\xab\xc5\xa2\x18\x1d>"
updated md5 b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x98\xec\xf8B~'
None
controlla checksum e aggiorna
hash da confrontare b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x98\xec\xf8B~'
updated md5 b'\xd4\x1d\x8c\xd9\x8f\x00\xb2\x04\xe9\x80\t\x98\xec\xf8B~'
None
controlla checksum e aggiorna
hash d